In [1]:
from pyspark.sql import SparkSession
from ucimlrepo import fetch_ucirepo
from pyspark.sql import functions as F
from pyspark.sql import Window
import matplotlib.pyplot as plt
import seaborn as sns
import os
print(f'{os.getcwd()}')
import pandas as pd

/home/jdwolfe/Spark-Projects/spark-learning-notebooks


# We will find the pitcher's who gave given up the most amount of one inning cycles

In [2]:
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

25/05/11 13:37:57 WARN Utils: Your hostname, JordansGaming resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/11 13:37:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/11 13:37:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
csv_headers = [
    "game id", "visiting team", "inning", "batting team", "outs", "balls", "strikes",
    "pitch sequence", "vis score", "home score", "batter", "batter hand", "res batter",
    "res batter hand", "pitcher", "pitcher hand", "res pitcher", "res pitcher hand",
    "catcher", "first base", "second base", "third base", "shortstop", "left field",
    "center field", "right field", "first runner", "second runner", "third runner",
    "event text", "leadoff flag", "pinchhit flag", "defensive position", "lineup position",
    "event type", "batter event flag", "ab flag", "hit value", "SH flag", "SF flag",
    "outs on play", "double play flag", "triple play flag", "RBI on play", "wild pitch flag",
    "passed ball flag", "fielded by", "batted ball type", "bunt flag", "foul flag",
    "hit location", "num errors", "1st error player", "1st error type", "2nd error player",
    "2nd error type", "3rd error player", "3rd error type", "batter dest",
    "runner on 1st dest", "runner on 2nd dest", "runner on 3rd dest", "play on batter",
    "play on runner on 1st", "play on runner on 2nd", "play on runner on 3rd",
    "SB for runner on 1st flag", "SB for runner on 2nd flag", "SB for runner on 3rd flag",
    "CS for runner on 1st flag", "CS for runner on 2nd flag", "CS for runner on 3rd flag",
    "PO for runner on 1st flag", "PO for runner on 2nd flag", "PO for runner on 3rd flag",
    "Responsible pitcher for runner on 1st", "Responsible pitcher for runner on 2nd",
    "Responsible pitcher for runner on 3rd", "New Game Flag", "End Game Flag",
    "Pinch-runner on 1st", "Pinch-runner on 2nd", "Pinch-runner on 3rd",
    "Runner removed for pinch-runner on 1st", "Runner removed for pinch-runner on 2nd",
    "Runner removed for pinch-runner on 3rd", "Batter removed for pinch-hitter",
    "Position of batter removed for pinch-hitter", "Fielder with First Putout",
    "Fielder with Second Putout", "Fielder with Third Putout", "Fielder with First Assist",
    "Fielder with Second Assist", "Fielder with Third Assist", "Fielder with Fourth Assist",
    "Fielder with Fifth Assist", "event num"
]

full_df = None

for file in os.listdir(f'{os.getcwd()}/data/mlb_data'):
    file_path = f'{os.getcwd()}/data/mlb_data/{file}'
    print(f'Processing {file_path}')
    df = spark.read.csv(file_path, header=False, inferSchema=True).toDF(*csv_headers)

    if full_df is None:
        full_df = df
    else:
        full_df = full_df.unionByName(df)

full_df.show()


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1970s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_2000s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1960s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1950s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1980s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1990s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_2020s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_2010s_pbp.csv


25/05/11 13:39:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+-------------+------+------------+----+-----+-------+--------------+---------+----------+--------+-----------+----------+---------------+--------+------------+-----------+----------------+--------+----------+-----------+----------+---------+----------+------------+-----------+------------+-------------+------------+------------+------------+-------------+------------------+---------------+----------+-----------------+-------+---------+-------+-------+------------+----------------+----------------+-----------+---------------+----------------+----------+----------------+---------+---------+------------+----------+----------------+--------------+----------------+--------------+----------------+--------------+-----------+------------------+------------------+------------------+--------------+---------------------+---------------------+---------------------+-------------------------+-------------------------+-------------------------+-------------------------+-----------------

In [4]:
full_df.printSchema()

root
 |-- game id: string (nullable = true)
 |-- visiting team: string (nullable = true)
 |-- inning: integer (nullable = true)
 |-- batting team: integer (nullable = true)
 |-- outs: integer (nullable = true)
 |-- balls: integer (nullable = true)
 |-- strikes: integer (nullable = true)
 |-- pitch sequence: string (nullable = true)
 |-- vis score: integer (nullable = true)
 |-- home score: integer (nullable = true)
 |-- batter: string (nullable = true)
 |-- batter hand: string (nullable = true)
 |-- res batter: string (nullable = true)
 |-- res batter hand: string (nullable = true)
 |-- pitcher: string (nullable = true)
 |-- pitcher hand: string (nullable = true)
 |-- res pitcher: string (nullable = true)
 |-- res pitcher hand: string (nullable = true)
 |-- catcher: string (nullable = true)
 |-- first base: string (nullable = true)
 |-- second base: string (nullable = true)
 |-- third base: string (nullable = true)
 |-- shortstop: string (nullable = true)
 |-- left field: string (nulla

In [5]:
full_df.count()

11769225

In [8]:
full_df = full_df.withColumn('Season', F.substring(F.col('game id'), 4, 4).cast('int'))

full_df.show()

+------------+-------------+------+------------+----+-----+-------+--------------+---------+----------+--------+-----------+----------+---------------+--------+------------+-----------+----------------+--------+----------+-----------+----------+---------+----------+------------+-----------+------------+-------------+------------+------------+------------+-------------+------------------+---------------+----------+-----------------+-------+---------+-------+-------+------------+----------------+----------------+-----------+---------------+----------------+----------+----------------+---------+---------+------------+----------+----------------+--------------+----------------+--------------+----------------+--------------+-----------+------------------+------------------+------------------+--------------+---------------------+---------------------+---------------------+-------------------------+-------------------------+-------------------------+-------------------------+-----------------

In [10]:
full_df.select(
    F.min(
        F.col('Season')
    ).alias('Min Season of Data'),

    F.max(
        F.col('Season')
    ).alias('Max Season of Data'),
).distinct().show()

+------------------+------------------+
|Min Season of Data|Max Season of Data|
+------------------+------------------+
|              1950|              2024|
+------------------+------------------+



In [ ]:
pandas_full = pd.DataFrame()
for file in os.listdir(f'{os.getcwd()}/data/mlb_data'):
    file_path = f'{os.getcwd()}/data/mlb_data/{file}'
    print(f'Processing {file_path}')
    df = pd.read_csv(file_path)

    pandas_full = pd.concat([pandas_full, df])


print(pandas_full.head())

Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1970s_pbp.csv


/tmp/ipykernel_1219/2705840558.py:5: DtypeWarning: Columns (7,50,64,65,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_2000s_pbp.csv


/tmp/ipykernel_1219/2705840558.py:5: DtypeWarning: Columns (63,64,65,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1960s_pbp.csv


/tmp/ipykernel_1219/2705840558.py:5: DtypeWarning: Columns (7,50,64,65,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1950s_pbp.csv


/tmp/ipykernel_1219/2705840558.py:5: DtypeWarning: Columns (7,64,65,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


### ^^ Pandas was taking almost 10 min here to process similar data to pyspark it didnt finish so I skipped it

In [6]:
pitcher_inning_cycles = full_df.groupby('pitcher', 'game id', 'inning').agg(
    F.sum(F.when(F.col('event type') == 20, 1).otherwise(0)).alias('singles'),
    F.sum(F.when(F.col('event type') == 21, 1).otherwise(0)).alias('doubles'),
    F.sum(F.when(F.col('event type') == 22, 1).otherwise(0)).alias('triples'),
    F.sum(F.when(F.col('event type') == 23, 1).otherwise(0)).alias('home runs')
)

pitcher_inning_cycles \
    .filter(
        (F.col('singles') > 0) &
        (F.col('doubles') > 0) &
        (F.col('triples') > 0) &
        (F.col('home runs') > 0)
    ) \
    .groupby('pitcher') \
    .agg(
    F.count('pitcher').alias('count')
).sort('count', ascending=False ) \
    .show()




25/05/11 13:41:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:41:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------+-----+
| pitcher|count|
+--------+-----+
|jacke001|    6|
|briln101|    4|
|lynnl001|    4|
|millk004|    3|
|forsb001|    3|
|davik002|    3|
|cardd101|    3|
|paulf001|    3|
|frieb102|    3|
|owenj101|    3|
|pappm101|    3|
|gullb001|    3|
|wegmb001|    3|
|lackj001|    3|
|smitm002|    3|
|radkb001|    3|
|garlj001|    3|
|mlicd001|    3|
|olivd001|    3|
|chatt001|    3|
+--------+-----+
only showing top 20 rows



## Cycles by Players Leaders

In [11]:
full_df.groupby('batter', 'game id').agg(
    F.sum(F.when(F.col('event type') == 20, 1).otherwise(0)).alias('singles'),
    F.sum(F.when(F.col('event type') == 21, 1).otherwise(0)).alias('doubles'),
    F.sum(F.when(F.col('event type') == 22, 1).otherwise(0)).alias('triples'),
    F.sum(F.when(F.col('event type') == 23, 1).otherwise(0)).alias('home runs')
    ) \
    .filter(

        (F.col('singles') > 0) &
        (F.col('doubles') > 0) &
        (F.col('triples') > 0) &
        (F.col('home runs') > 0)

    ).groupby('batter') \
    .agg(
    F.count('batter').alias('count')
).sort('count', ascending=False ) \
    .show()

25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on

+--------+-----+
|  batter|count|
+--------+-----+
|yelic001|    3|
|belta001|    3|
|turnt001|    3|
|gomec002|    2|
|arenn001|    2|
|olerj001|    2|
|boyek101|    2|
|watsb001|    2|
|speic001|    2|
|bretg001|    2|
|hilla001|    2|
|freef001|    2|
|wilkb002|    2|
|fregj101|    2|
|cuddm001|    2|
|whitf001|    2|
|cedec001|    2|
|mcdoo001|    1|
|dejei001|    1|
|teixm001|    1|
+--------+-----+
only showing top 20 rows

